In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!pip install pyLDAvis

In [ ]:
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
!pip install --upgrade pythainlp

In [ ]:
import pythainlp

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/8101 Customer Analytics/TopicModeling.xlsx')

In [ ]:
df.tail()

,No.,Comment
10,11,สร้างระบบสั่งเมนูล่วงหน้าออนไลน์รอ ไปถึงแล้วได...
11,12,ความสะอาด ความหลากหลายของอาหาร และแต่ละสาขาควร...
12,13,ไม่ต้องให้ขึ้นที่1แต่ให้ขายดีที่สุด... อย่างแข...
13,14,เอางบโฆษณา มาแจกให้กินฟรีครับ ช่วงเวลาหนึ่ง คน...
14,15,แตกแบรนด์ เก็บลูกค้าให้ครบทุก Segment\nนำเข้าว...


#Tokenize


In [ ]:
stopwords = list(pythainlp.corpus.thai_stopwords())

In [ ]:
removed_words = ['',' ','  ','\n', '(', ')','-','=',' ','/','.']

In [ ]:
screening_words = stopwords + removed_words

In [ ]:
def tokenize_with_space(sentence):
  merged = ''
  words = pythainlp.word_tokenize(str(sentence), engine='newmm')
  for word in words:
    if word not in screening_words:
      merged = merged + ',' + word
  return merged[1:]

In [ ]:
df['Comment_token'] = df['Comment'].apply(lambda x: tokenize_with_space(x))

In [ ]:
df.head()

,No.,Comment,Comment_token
0,1,เปิด24ชม.\nแบ่ง3ช่วงเวลา 3ธีม ตั้งชื่อตามมณฑล ...,"24,ชม.,แบ่ง,3,ช่วงเวลา,3,ธีม,ตั้งชื่อ,มณฑล,เช้..."
1,2,เบื่อเรื่องการรอคิวที่นานเกินไป คือเข้าใจว่าต้...,"เบื่อ,เรื่อง,รอ,คิว,เกินไป,รอ,ร้าน,รอ,ก,ไก่,ล้..."
2,3,มีน้ำจิ้มให้เลือกมากกว่านี้ค่ะ,"น้ำจิ้ม,เลือก"
3,4,เป็นเบอร์1 แล้ว ก็มีวันร่วง มองที่ความยั่งยืนด...,"เบอร์,1,ร่วง,ความยั่งยืน,ดีกว่า,เหมือน,mk,iber..."
4,5,ทำน้ำซุปหมาล่าให้เค็มน้อยลงค่ะ,"ทำ,น้ำ,ซุป,หมา,ล่า,เค็ม,น้อยลง"


#CREATE DICTIONNARY

In [ ]:
documents = df['Comment_token'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [ ]:
print(dictionary.token2id.keys())

dict_keys(['%', '24', '3', '30', 'MK', 'ขาย', 'ความหลากหลาย', 'ชนะ', 'ชม.', 'ช่วงเวลา', 'ตั้งชื่อ', 'ตี้', 'ต่างกัน', 'ธีม', 'บ่าย', 'ปักกิ่ง', 'มณฑล', 'ยี่', 'รูป', 'ร้าน', 'หัวค่ำ', 'อารมณ์', 'อาหาร', 'เช้า', 'เซียงไฮ้', 'เทคโน', 'เวลาเปิด', 'เสฉวน', 'แบ่ง', 'แสง', 'โล', 'ก', 'กระจาย', 'กระจุก', 'คน', 'คิว', 'จา', 'ตัว', 'ทาน', 'รอ', 'ล้าน', 'สาขา', 'อี้', 'เกินไป', 'เค้า', 'เดี๋ยวนี้', 'เบื่อ', 'เรื่อง', 'เหมือนกัน', 'แทน', 'ไก่', 'น้ำจิ้ม', 'เลือก', '1', 'brand', 'iberry', 'mk', 'segment', 'ความยั่งยืน', 'ดีกว่า', 'ร่วง', 'ออกมา', 'เบอร์', 'เหมือน', 'แตก', 'ซุป', 'ทำ', 'น้อยลง', 'น้ำ', 'ล่า', 'หมา', 'เค็ม', 'ความสะอาด', 'ชอบ', 'บริการ', 'บางประการ', 'มาตรฐาน', 'อีกแล้ว', 'ใจ', 'จอง', 'บุฟเฟต์', 'ปี', 'ออนไลน์', 'ตัวเอง', 'ต้นทุน', 'รักษา', 'ลด', 'วัตถุดิบ', 'อร่อย', 'ให้ได้', 'ความสะดวก', 'ที่จอดรถ', 'ที่นั่ง', 'ระบบ', 'ลูกค้า', 'สำหรับ', 'ค่าบริการ', 'นาที', 'ฟรี', 'เดือด', 'เตา', 'ขับรถ', 'จุ่ม', 'ล่วงหน้า', 'สร้าง', 'สั่ง', 'เมนู', 'ไม่ต้อง', 'กิน', 'ควรจะ', 'รสชาติ', 'สัก', '..

In [ ]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

#TOPIC MODELING

In [ ]:
num_topics = 4
chunksize = 4000
passes = 20
iterations = 50
eval_every = 1

temp = dictionary[0]
id2word = dictionary.id2token

model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                              alpha='auto', eta='auto', \
                              iterations=iterations, num_topics=num_topics, \
                              passes=passes, eval_every=eval_every)

In [ ]:
vis = pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary, mds='mmds')
vis

/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.116995 -0.077018       1        1  39.958234
3     -0.047157  0.126229       2        1  22.439855
0      0.103112 -0.095546       3        1  22.014839
1      0.061040  0.046334       4        1  15.587072, topic_info=          Term      Freq     Total Category  logprob  loglift
142          ​  3.000000  3.000000  Default  30.0000  30.0000
53           1  3.000000  3.000000  Default  29.0000  29.0000
39          รอ  5.000000  5.000000  Default  28.0000  28.0000
13         ธีม  2.000000  2.000000  Default  27.0000  27.0000
2            3  2.000000  2.000000  Default  26.0000  26.0000
..         ...       ...       ...      ...      ...      ...
74      บริการ  0.590351  1.608934   Topic4  -4.1898   0.8561
72   ความสะอาด  0.590437  2.167648   Topic4  -4.1896   0.5582
53           1  0.590398  3.864329   Topic4  -4.1897  -0.0200
89      ให้ได้  0.590397  1.741212   Topic4  -4.1897   0.7772
94      ลูกค้า  0.590396  5.089631   Topic4  -4.1897  -0.2954

[180 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
0         3  0.883786        %
112       2  0.588077      ...
53        2  0.776332        1
53        4  0.258777        1
1         3  0.883786       24
...     ...       ...      ...
107       2  0.545633  ไม่ต้อง
141       2  0.879812      ไร้
166       1  0.788089      ไหล
142       2  0.884431        ​
143       2  0.879812       😁😊

[158 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 1, 2])

#PREDICT TOPICS

In [ ]:
model.show_topic(0)

[('ธีม', 0.03299199),
 ('3', 0.032990314),
 ('อาหาร', 0.03298758),
 ('ชนะ', 0.022841632),
 ('บ่าย', 0.022840932),
 ('เช้า', 0.022840917),
 ('ช่วงเวลา', 0.022840703),
 ('หัวค่ำ', 0.022840545),
 ('รสชาติ', 0.022838395),
 ('ความหลากหลาย', 0.022838224)]

In [ ]:
model.show_topic(1)

[('มาตรฐาน', 0.027273191),
 ('แตก', 0.027271865),
 ('วัตถุดิบ', 0.027271843),
 ('ความสะอาด', 0.015151972),
 ('segment', 0.0151509745),
 ('Segment', 0.015150972),
 ('mk', 0.015150969),
 ('iberry', 0.015150966),
 ('เบอร์', 0.015150966),
 ('ร่วง', 0.015150966)]

In [ ]:
model.show_topic(2)

[('รอ', 0.050522953),
 ('ลูกค้า', 0.036585636),
 ('คน', 0.022648215),
 ('ฟรี', 0.022648124),
 ('ร้าน', 0.015679495),
 ('คิว', 0.015679479),
 ('สาขา', 0.015679466),
 ('แจก', 0.015679454),
 ('จ่าย', 0.01567945),
 ('ทาน', 0.015679441)]

In [ ]:
model.show_topic(3)

[('\u200b', 0.052760396),
 ('1', 0.05276033),
 ('ดี', 0.03266038),
 ('...', 0.022611303),
 ('ชนะ', 0.012563616),
 ('ปี', 0.012563414),
 ('บุฟเฟต์', 0.012563407),
 ('คิว', 0.012563386),
 ('จอง', 0.012563379),
 ('ออนไลน์', 0.012563377)]

In [ ]:
df['topics'] = df['Comment_token'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Comment_token'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [ ]:
df

,No.,Comment,Comment_token,topics,score
0,1,เปิด24ชม.\nแบ่ง3ช่วงเวลา 3ธีม ตั้งชื่อตามมณฑล ...,"24,ชม.,แบ่ง,3,ช่วงเวลา,3,ธีม,ตั้งชื่อ,มณฑล,เช้...",0,0.996316
1,2,เบื่อเรื่องการรอคิวที่นานเกินไป คือเข้าใจว่าต้...,"เบื่อ,เรื่อง,รอ,คิว,เกินไป,รอ,ร้าน,รอ,ก,ไก่,ล้...",2,0.995371
2,3,มีน้ำจิ้มให้เลือกมากกว่านี้ค่ะ,"น้ำจิ้ม,เลือก",0,0.933871
3,4,เป็นเบอร์1 แล้ว ก็มีวันร่วง มองที่ความยั่งยืนด...,"เบอร์,1,ร่วง,ความยั่งยืน,ดีกว่า,เหมือน,mk,iber...",1,0.988811
4,5,ทำน้ำซุปหมาล่าให้เค็มน้อยลงค่ะ,"ทำ,น้ำ,ซุป,หมา,ล่า,เค็ม,น้อยลง",2,0.983770
5,6,ความสะอาด มาตรฐาน การบริการ จากใจคนเคยชอบแต่ด้...,"ความสะอาด,มาตรฐาน,บริการ,ใจ,คน,ชอบ,บางประการ,อ...",1,0.983342
6,7,จองคิวออนไลน์ ต้องมาก่อนเลยครับ\nบุฟเฟต์ปีละ 1...,"จอง,คิว,ออนไลน์,บุฟเฟต์,ปี,1",2,0.010740
7,8,รักษามาตรฐานตัวเองให้ได้ และไม่เอาวัตถุดิบ ไม่...,"รักษา,มาตรฐาน,ตัวเอง,ให้ได้,วัตถุดิบ,อร่อย,ลด,...",1,0.983342
8,9,ระบบการจองคิว ที่นั่งรอ ที่จอดรถ หลายๆสาขาไม่ม...,"ระบบ,จอง,คิว,ที่นั่ง,รอ,ที่จอดรถ,สาขา,ความสะดว...",2,0.988551
9,10,คิดค่าบริการเป็นนาที ทุกอย่างฟรีหมด แล้วทำเตาใ...,"ค่าบริการ,นาที,ฟรี,ทำ,เตา,เดือด",2,0.981145
